# Cosmetics EDA

In [10]:
import importlib

import analysis_code.myevaluation
importlib.reload(analysis_code.myevaluation)
import analysis_code.myevaluation as myevaluation

import analysis_code.myutils
importlib.reload(analysis_code.myutils)
import analysis_code.myutils as myutils

import analysis_code.mypytable
importlib.reload(analysis_code.mypytable)
from analysis_code.mypytable import MyPyTable 

import analysis_code.myclassifiers
importlib.reload(analysis_code.myclassifiers)
from analysis_code.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier

import tabulate
from tabulate import tabulate

## Loading the Datasets for Comparison

In [ ]:
raw_data = MyPyTable()
raw_data2 = MyPyTable()
raw_data = raw_data.load_from_file("california_cosmetics.csv")
raw_data2 = raw_data2.load_from_file("cosmetics_kaggle.csv")
data_joined = raw_data.perform_inner_join(raw_data2, ["BrandName"])
duplicates = data_joined.find_duplicates(["BrandName"])
data_joined.drop_rows(duplicates)
data_joined.save_to_file("cosmetics_clean.csv")
row, col = data_joined.get_shape()
print("number of rows ", row)
print("number of cols ", col)

number of rows  13
number of cols  32


#### Based on the lack of matching columns we decided to move away from using the cosmetics kaggle data set and instead are splitting the data set in order to test our classifiers. The size of our dataset is sufficient due to it being more than 100,000 entries that need to be sorted through.